In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4

pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [2]:
df6 = pd.read_csv(cache_dir+'article_details_v6.tsv', sep='\t')
df7 = pd.read_csv(cache_dir+'article_details_v7.tsv', sep='\t').fillna(0).drop('Alexa Rank', axis=1)
df7['Author Signature'] = df6['authors'].apply(lambda x: len(eval(x)) != 0)


In [3]:
for a in ['#Likes', '#Replies', 'Title Clickbaitness', 'Betweenness Centrality', 'Degree Centrality', 
          'In Degree Centrality', 'Out Degree Centrality', '#Retweets', 'Tweets Shelf Life', '#Users Countries', 
          '#Followers', '#Users Friends', '#Users Tweets', 'STS', 'Readability', '#Quotes', '#Person Quotes', 
          '#Scientific Mentions', '#Weasel Quotes', 'Personalized PageRank', 'Article Word Count']:
    
    df7[a] = pd.cut(df7[a], 5, labels=False)
    df7[a] = df7[a].apply(lambda x: (int(x)+1) * '★' + (5-(int(x)+1)) * '☆')

In [4]:
for a in ['Replies Polarity', 'Replies Subjectivity', 'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity', 
          'Title Subjectivity', 'Title Polarity']:
    
    df7[a] = pd.cut(df7[a], 5, labels=False)
    df7[a] = df7[a].apply(lambda x: {0:'🙁🙁', 1:'🙁', 2:'😐', 3:'🙂', 4:'🙂🙂'}[x])


In [5]:
df7['Author Signature'] = df7['Author Signature'].apply(lambda x: '✔' if x==True else '✘')

In [6]:
df7.to_csv(cache_dir+'article_details_v8.tsv', sep='\t', index=None)

In [7]:
df7 = pd.read_csv(cache_dir+'alc_tob_caf.tsv', sep='\t').merge(df7, left_on='article', right_on='url')
dfa = pd.read_csv(cache_dir+'alexa.tsv', sep='\t')

In [8]:
df7 = df7.merge(dfa)
df7['Alexa Rank'] = pd.cut(df7['Alexa Rank'], 5, labels=False)

In [9]:
df7['Alexa Rank'] = 5-df7['Alexa Rank']

In [10]:
df7['Alexa Rank'] = df7['Alexa Rank'].apply(lambda x: int(x) * '★' + (5-(int(x))) * '☆')
df7 = df7.drop('url', axis=1)

In [11]:
df7

,article,title,paper1,title1,paper2,title2,paper3,title3,#Likes,#Replies,...,Readability,Title Subjectivity,Title Polarity,#Quotes,#Person Quotes,#Scientific Mentions,#Weasel Quotes,Author Signature,Article Word Count,Alexa Rank
0,http://fortune.com/2018/01/04/philip-morris-qu...,Marlboro Maker Philip Morris Says It Wants to ...,http://www.cdc.gov/tobacco/data_statistics/fac...,Smoking & Tobacco Use - Fast Facts,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
1,http://humanprogress.org/article.php?p=503,How Alcohol and Caffeine Helped Create Civiliz...,http://www.cdc.gov/alcohol/fact-sheets/alcohol...,Fact Sheets - Alcohol Use and Your Health,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★★★★
2,http://mentalfloss.com/article/526832/sorry-ba...,"Sorry, Barflies: Dry January Isn't a Fix For H...",http://pubs.niaaa.nih.gov/publications/arh25-2...,"Sleep, Sleepiness, and Alcohol Use",NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙂,🙁,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
3,http://uk.businessinsider.com/indoor-smoking-b...,A 15-year study finds indoor smoking bans real...,http://www.sciencedirect.com/science/article/p...,Indoor tobacco legislation is associated with ...,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
4,http://uk.businessinsider.com/philip-morris-e-...,One of the world's largest tobacco companies h...,http://www.cdc.gov/tobacco/data_statistics/fac...,Smoking & Tobacco Use – Tobacco-Related Mortality,http://www.cdc.gov/tobacco/campaign/tips/resou...,Tips From Former Smokers – Burden of Tobacco U...,http://www.nature.com/nature/journal/v247/n544...,Temperature distribution inside a burning ciga...,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙂,🙁,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
5,http://vantagepointrecovery.com/5-reasons-seek...,5 Reasons You Should Seek Support When Quittin...,http://www.niaaa.nih.gov/alcohol-health/overvi...,Alcohol Use Disorder,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★☆☆☆
6,http://winefolly.com/update/alcohol-your-brain...,Study Offers A New Look at Alcohol and Your Brain,http://www.niaaa.nih.gov/alcohol-health/overvi...,Drinking Levels Defined,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★★★★
7,http://www.drugaddictionnow.com/2016/11/28/tre...,Treating a “dual diagnosis” of addiction and m...,http://www.samhsa.gov/disorders/co-occurring,Co-occurring Disorders,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★☆☆☆☆
8,http://www.esquire.com/food-drink/food/a167564...,California Might Be Slapping a Cancer Warning ...,http://www.cancer.gov/about-cancer/causes-prev...,Acrylamide and Cancer Risk,NaN,NaN,NaN,NaN,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
9,http://www.healthline.com/nutrition/top-13-evi...,"13 Health Benefits of Coffee, Based on Science",http://www.ncbi.nlm.nih.gov/pubmed/1356551,Caffeine and the central nervous system: mecha...,http://www.sciencedirect.com/science/article/p...,Coffee Consumption and Risk of Liver Cancer: A...,http://www.ncbi.nlm.nih.gov/pubmed/15657469/,Effects of caffeine ingestion on exercise test...,★☆☆☆☆,★☆☆☆☆,...,★★★★★,🙁🙁,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★★☆☆☆,✘,★☆☆☆☆,★★★★★


In [14]:
#df7['Alexa Rank'] = df7['url'].apply(alexa)
df7.to_csv(cache_dir+'alc_tob_caf_v2.tsv', sep='\t', index=None)

In [ ]:
import subprocess
import os
import string

def is_clickbait(title):
    
    title = ''.join([c for c in title if c in string.printable])
    os.chdir('../lib/clickbait')
    out, err = subprocess.Popen(['venv/bin/python', 'src/detect.py', title], stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    os.chdir('../../src')
    return float(re.findall('\d*\.?\d+', str(out))[0])

In [ ]:
outlet1 = 'dailymail.co.uk'
outlet2 = 'nytimes.com'
outlet3 = 'theatlantic.com'
outlet4 = 'washingtonpost.com'

G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v3.tsv')
pagerank = nx.pagerank(G.reverse())
betweenness_centrality = nx.betweenness_centrality(G)
degree_centrality = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)

In [ ]:
outlet = outlet3
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')


df[df.apply(lambda x: analyze_url(x['url'])[0]==outlet, axis=1)].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+outlet+'.tsv', cache_dir+outlet+'.tsv')
df = pd.read_csv(cache_dir+outlet+'.tsv', sep='\t').fillna(0)

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(x).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(x).polarity)
df['title_clickbaitness'] = df['title'].apply(is_clickbait)

df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = df['authors'].apply(lambda x: len(eval(x)) != 0)

df['pageRank'] = df['url'].apply(lambda x: pagerank[x])
df['betweenness_centrality'] = df['url'].apply(lambda x: betweenness_centrality[x])
df['degree_centrality'] = df['url'].apply(lambda x: degree_centrality[x])
df['in_degree_centrality'] = df['url'].apply(lambda x: in_degree_centrality[x])
df['out_degree_centrality'] = df['url'].apply(lambda x: out_degree_centrality[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df = df[['url', 'likes', 'replies_count', 'title_clickbaitness',
    'betweenness_centrality', 'degree_centrality', 'in_degree_centrality', 'out_degree_centrality',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']]

df.columns = ['url', '#Likes', '#Replies', 'Title Clickbaitness',
    'Betweenness Centrality', 'Degree Centrality', 'In Degree Centrality', 'Out Degree Centrality',
    'Replies Polarity', 'Replies Subjectivity', '#Retweets',
    'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity',
    'Tweets Shelf Life', '#Users Countries', '#Followers',
    '#Users Friends', '#Users Tweets', 'STS', 'Readability',
    'Title Subjectivity', 'Title Polarity', '#Quotes',
    '#Person Quotes', '#Scientific Mentions', '#Weasel Quotes',
    'Author Signature', 'Personalized PageRank', 'Article Word Count', 'Alexa Rank']

df.to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

In [ ]:
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+'article_details_v5.tsv', cache_dir+'article_details_v6.tsv')
df = pd.read_csv(cache_dir+'article_details_v6.tsv', sep='\t').fillna(0)

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(str(x)))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(str(x)).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(str(x)).polarity)

In [ ]:
df['title_clickbaitness'] = df['title'].apply(is_clickbait)

In [ ]:
df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = ~(df['authors'].isnull() | (df['authors'] == ''))

df['pageRank'] = df['url'].apply(lambda x: pagerank[x])
df['betweenness_centrality'] = df['url'].apply(lambda x: betweenness_centrality[x])
df['degree_centrality'] = df['url'].apply(lambda x: degree_centrality[x])
df['in_degree_centrality'] = df['url'].apply(lambda x: in_degree_centrality[x])
df['out_degree_centrality'] = df['url'].apply(lambda x: out_degree_centrality[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df = df[['url', 'likes', 'replies_count', 'title_clickbaitness',
    'betweenness_centrality', 'degree_centrality', 'in_degree_centrality', 'out_degree_centrality',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']]

df.columns = ['url', '#Likes', '#Replies', 'Title Clickbaitness',
    'Betweenness Centrality', 'Degree Centrality', 'In Degree Centrality', 'Out Degree Centrality',
    'Replies Polarity', 'Replies Subjectivity', '#Retweets',
    'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity',
    'Tweets Shelf Life', '#Users Countries', '#Followers',
    '#Users Friends', '#Users Tweets', 'STS', 'Readability',
    'Title Subjectivity', 'Title Polarity', '#Quotes',
    '#Person Quotes', '#Scientific Mentions', '#Weasel Quotes',
    'Author Signature', 'Personalized PageRank', 'Article Word Count', 'Alexa Rank']

df.to_csv(cache_dir+'article_details_v6.tsv', sep='\t', index=None)

In [ ]:
df = pd.concat([pd.read_csv(cache_dir+outlet2+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet3+'.tsv', sep='\t')])
y = df[['url']].values
X = np.array(df.drop('url', axis=1).values, dtype=np.float32)
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)

from sklearn.feature_selection import f_classif
_, result = f_classif(X,y)

d = {c:r for c, r in zip(df.columns[1:], result)}
df = pd.DataFrame(sorted(d.items(), key=lambda kv: kv[1]))

df[0] = df.apply(lambda x: x[0]+'**' if float(x[1])<.005 else x[0]+'*' if float(x[1])<.01 else x[0], axis=1)

df23 = df.copy()

In [ ]:
df = pd.concat([df12, df13, df23, df123], axis=1).head(15)[0]
df.columns = ['NYTimes-DailyMail', 'TheAtlantic-DailyMail', 'NYTimes-TheAtlantic', 'NYTimes-TheAtlantic-DailyMail']

In [ ]:
print(df.to_latex(index=False))